# Miniproyecto

## Descripción

La tarea se divide en dos sub-tareas: crear un script en Python que genere un reporte en formato texto plano con un resumen de peticiones HTTP procesadas por el servidor, y preparar un playbook de Ansible que automatice los pasos necesarios para desplegar dicho script y que se ejecute periódicamente en una máquina virtual Debian 10.

### Script de reporte

Las tareas que debe realizar el script son las siguientes:

* Leer el fichero `/var/log/nginx/access.log`
* Extraer el primer campo de cada línea, el que corresponde a la IP de origen de cada petición
* Contar cuántas peticiones han venido de cada IP única. **Pista:** la clase [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) viene muy bien para esto
* Volcar los datos obtenidos en un fichero. La ruta de este fichero se deja a elección del alumno, pero se sugiere `/opt/report.txt`. Ejemplo de `report.txt`:

```
127.0.0.1 265
192.168.122.1 117
```

Nótese que las IPs y su correspondiente contador de peticiones están ordenadas de mayor a menor por número de peticiones. La separación entre IP y el contador puede ser cualquier símbolo o secuencia de símbolos, pero se sugiere un espacio o tabulador por sencillez.

### Playbook de Ansible

Una vez el mencionado script funciona adecuadamente por sí solo, hay que crear un playbook de Ansible que realice las siguientes tareas en una máquina virtual Debian 10 recién instalada:

* Instalar nginx
* Copiar el script de reporte a la máquina remota; como ruta se sugiere `/opt/nginx-metrics-reporter.py`
* Activar un [timer](https://www.freedesktop.org/software/systemd/man/systemd.timer.html) de systemd que ejecute dicho script cada 15 minutos, actualizando de esta forma los contenidos de `/opt/report.txt` periódicamente

Hay que intentar utilizar módulos específicos de Ansible y reducir al máximo el uso de comandos de shell _ad-hoc_. Ejemplos de módulos relevantes para la tarea: [apt](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/apt_module.html), [copy](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/copy_module.html), [systemd](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/systemd_module.html)

### Timer de systemd

Ésta es quizás la parte más compleja. Requiere un poco de investigación acerca de cómo funcionan estos componentes: en pocas palabras, ejecutan un _servicio_ asociado con una periodicidad configurable. Recordemos que un servicio de systemd representa habitualmente un programa que se ejecuta "de fondo", sin interactuar directamente con el usuario, y que permanece en ejecución desde que la máquina arranca hasta que se apaga.

Sin embargo, el concepto de servicio entendido por systemd es más amplio y puede también representar programas que simplemente se ejecutan, hacen su trabajo, y terminan. Estos tipos de servicios están especificados [aquí](https://wiki.archlinux.org/index.php/Systemd#Service_types) o [aquí](https://www.freedesktop.org/software/systemd/man/systemd.service.html#Type=), y probablemente el tipo que más nos interesa es el `oneshot`. Es altamente recomendable tratar de "envolver" el script de reporte en un servicio de systemd y comprobar que funciona antes de atacar el timer.

Ejemplo mínimo de servicio:

```ini
[Unit]
Description=Generate nginx access log report

[Service]
Type=oneshot
ExecStart=/usr/bin/python3 /opt/nginx-metrics-reporter.py
```

Estos ficheros tienen que tener extensión `.service` y pueden ser depositados en multitud de rutas en las que systemd busca servicios, pero se recomienda `/usr/lib/systemd/system`, tras lo cual hay que ejecutar el comando `systemctl daemon-reload` para que re-escanee el sistema en busca de cambios en los ficheros de definición de servicios (en el módulo de Ansible hay un parámetro `daemon_reload` que tiene la funcionalidad equivalente ([referencia](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/systemd_module.html#parameter-daemon_reload)))

Para comprobar que el servicio funciona correctamente, podemos ejecutarlo con `systemctl start nginx-metrics-reporter` y comprobar el estado con `systemctl status nginx-metrics-reporter`; cualquier problema de ejecución será reportado ahí.

Una vez hemos comprobado que el servicio funciona correctamente, debemos crear el timer. Un fichero `<nombre>.timer` controla la periodicidad con la que se lanza la tarea `<nombre>.service` asociada (**importante:** `<nombre>` debe ser el mismo en ambos casos, ya que así es como systemd establece la asociación entre ellos. En este ejemplo estamos usando `nginx-metrics-reporter` como nombre). Un fichero `.timer` es similar en cuanto a sintaxis y estructura a un `.service`; recomiendo estudiar [este](https://wiki.archlinux.org/index.php/Systemd/Timers) artículo de la wiki de Arch Linux para familiarizarse con su estructura. El objetivo en este caso es, una vez más, averiguar cómo hacer que dicho timer lance el servicio `nginx-metrics-reporter.service` cada 15 minutos. En cuanto a Ansible, se puede observar cómo activar un timer utilizando el módulo de systemd en la [sección de ejemplos](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/systemd_module.html#examples) (especialmente la task llamada _"Enable a timer for dnf-automatic"_).

## Criterios de evaluación

En orden de importancia, se evaluará la entrega acorde a los siguientes puntos:

1. **<ins>Funcionalidad</ins>**: Después de ser ejecutado, se probará la efectividad del playbook abriendo un navegador dentro de la máquina virtual y otro en la máquina host; en ambos casos se harán peticiones al puerto 80 de la máquina virtual (asumiendo modo `bridged` para la configuración de red de VirtualBox). El script deberá generar el reporte correcto con los números de peticiones actualizados desde al menos 2 direcciones IP diferentes la siguiente vez que se ejecute para cumplir con este criterio.
2. **<ins>Eficiencia</ins>**: Se valorará el utilizar módulos específicos de Ansible allá donde sea posible, por encima de comandos _ad-hoc_ usando módulos como [shell](https://docs.ansible.com/ansible/latest/modules/shell_module.html) o [command](https://docs.ansible.com/ansible/latest/modules/command_module.html#command-module). Por ejemplo, si queremos instalar un paquete con `apt install <paquete>`, es mejor utilizar el módulo [apt](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/apt_module.html) que invocar el comando correspondiente con los mencionados `shell` o `command`
3. **<ins>Limpieza</ins>**: Se debe intentar que las _tasks_ de Ansible sean claras, directas, eviten redundancias y tengan un campo `name` descriptivo. Asimismo, el/los ficheros YAML deben seguir las reglas del formato y mantener la consistencia en cuanto a espaciado, etc.
4. **<ins>Extra-mile</ins>**: Agregar funcionalidades de Ansible que no hayan sido vistas en clase, allá donde tengan sentido: handlers, variables, roles, etc.

## Fecha límite de entrega

La tarea deberá ser entregada, a más tardar, el **domingo 20 de diciembre** a las **23:59 UTC+1**

## Forma de entrega

Se habilitará una tarea en nuestro Google Classroom a tal efecto para poder subir los ficheros finales.